In [5]:
# import
import os
from dotenv import load_dotenv

import openai

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)

from typing import Optional
from pydantic import BaseModel, Field
from typing import Sequence

In [2]:
# set OpenAI API key
load_dotenv(dotenv_path="../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
# set OpenAI API Key to langchain and set temperature to 0.9
chat = ChatOpenAI(model="gpt-4", openai_api_key=openai.api_key, temperature=0.9)

In [3]:
# 1. Getting Structured Output using "create_structured_output_chain"
# 1-1. pydantic class
class Person(BaseModel):
    # Identifying information about a person.
    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [4]:
prompt_msgs = [
    SystemMessage(
        content="You are a world class algorithm for extracting information in structured formats."
    ),
    HumanMessage(
        content="Use the given format to extract information from the following input:"
    ),
    HumanMessagePromptTemplate.from_template("{input}"),
    HumanMessage(content="Tips: Make sure to answer in the correct format"),
]
prompt = ChatPromptTemplate(messages=prompt_msgs)

chain = create_structured_output_chain(Person, chat, prompt, verbose=True)
chain.run("Sally is 13")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input:
Human: Sally is 13
Human: Tips: Make sure to answer in the correct format

> Finished chain.


Person(name='Sally', age=13, fav_food='Unknown')

In [7]:
# wrapper Pydantic class that takes a sequence of the original class.
class People(BaseModel):
    # Identifying information about all people in a text.
    people: Sequence[Person] = Field(..., description="The people in the text")

In [8]:
chain = create_structured_output_chain(People, chat, prompt, verbose=True)
chain.run(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally, so she's 23."
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input:
Human: Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally, so she's 23.
Human: Tips: Make sure to answer in the correct format

> Finished chain.


People(people=[Person(name='Sally', age=13, fav_food='unknown'), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food='unknown')])

In [9]:
# 1-2. json schema
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}

chain = create_structured_output_chain(json_schema, chat, prompt, verbose=True)
chain.run("Sally is 13")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input:
Human: Sally is 13
Human: Tips: Make sure to answer in the correct format

> Finished chain.


{'name': 'Sally', 'age': 13}

In [10]:
# 2. Creating generic OpenAI functions chain using "create_openai_fn_chain"
# 2-1. pydantic class
class OptionalFavFood(BaseModel):
    # Either a food or null.
    food: Optional[str] = Field(
        None,
        description="Either the name of a food or null. Should be null if the food isn't known.",
    )


def record_person(name: str, age: int, fav_food: OptionalFavFood) -> str:
    """
    Record some basic identifying information about a person.

    Args:
        name: The person's name.
        age: The person's age in years.
        fav_food: An OptionalFavFood object that either contains the person's favorite food or a null value. Food should be null if it's not known.
    """
    return f"Recording person {name} of age {age} with favorite food {fav_food.food}!"



In [11]:
chain = create_openai_fn_chain([record_person], chat, prompt, verbose=True)
chain.run(
    "The most important thing to remember about Tommy, my 12 year old, is that he'll do anything for apple pie."
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input:
Human: The most important thing to remember about Tommy, my 12 year old, is that he'll do anything for apple pie.
Human: Tips: Make sure to answer in the correct format

> Finished chain.


{'name': 'Tommy', 'age': 12, 'fav_food': {'food': 'apple pie'}}

In [12]:
def record_dog(name: str, color: str, fav_food: OptionalFavFood) -> str:
    """Record some basic identifying information about a dog.

    Args:
        name: The dog's name.
        color: The dog's color.
        fav_food: An OptionalFavFood object that either contains the dog's favorite food or a null value. Food should be null if it's not known.
    """
    return f"Recording dog {name} of color {color} with favorite food {fav_food}!"


chain = create_openai_fn_chain([record_person, record_dog], chat, prompt, verbose=True)
chain.run(
    "I can't find my dog Henry anywhere, he's a small brown beagle. Could you send a message about him?"
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input:
Human: I can't find my dog Henry anywhere, he's a small brown beagle. Could you send a message about him?
Human: Tips: Make sure to answer in the correct format

> Finished chain.


{'name': 'record_dog',
 'arguments': {'name': 'Henry', 'color': 'brown', 'fav_food': {'food': None}}}